NESTML active dendrite third-factor STDP synapse
==========================================

In this tutorial, a neuron and synapse model are defined in NESTML that are subsequently used in a network to perform learning, prediction and replay of sequences of items, such as letters, images or sounds [1].

<a name="introduction"></a>

Introduction
------------


In [1]:
%matplotlib inline

from typing import List, Optional

import matplotlib as mpl

mpl.rcParams['axes.formatter.useoffset'] = False
mpl.rcParams['axes.grid'] = True
mpl.rcParams['grid.color'] = 'k'
mpl.rcParams['grid.linestyle'] = ':'
mpl.rcParams['grid.linewidth'] = 0.5
mpl.rcParams['figure.dpi'] = 120
mpl.rcParams['figure.figsize'] = [8., 3.]
# plt.rcParams['font.size'] = 8
# plt.rcParams['legend.fontsize']= 6
# plt.rcParams['figure.figsize'] = fig_size
# plt.rcParams['savefig.dpi'] = 300
# plt.rcParams['font.family'] = 'sans-serif'
# plt.rcParams['lines.linewidth'] = 1
# plt.rcParams['text.usetex'] = False



import matplotlib.pyplot as plt
import nest
import numpy as np
import os
import random
import re

from pynestml.codegeneration.nest_code_generator_utils import NESTCodeGeneratorUtils
from pynestml.codegeneration.nest_tools import NESTTools


              -- N E S T --
  Copyright (C) 2004 The NEST Initiative

 Version: 3.6.0-post0.dev0
 Built: Feb  5 2024 05:39:31

 This program is provided AS IS and comes with
 NO WARRANTY. See the file LICENSE for details.

 Problems or suggestions?
   Visit https://www.nest-simulator.org

 Type 'nest.help()' to find out more about NEST.



/home/charl/.local/lib/python3.11/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


## Generating code with NESTML

We will take a simple current-based integrate-and-fire model with alpha-shaped postsynaptic response kernels (``iaf_psc_alpha``) as the basis for our modifications. First, let's take a look at this base neuron without any modifications.

We will use a helper function to generate the C++ code for the models, build it as a NEST extension module, and load the module into the kernel. Because NEST does not support un- or reloading of modules at the time of writing, we implement a workaround that appends a unique number to the name of each generated model, for example, "iaf_psc_alpha_3cc945f". The resulting neuron model name is returned by the function, so we do not have to think about these internals.

In [2]:
%pdb
# codegen_opts = {"neuron_synapse_pairs": [{"neuron": "iaf_psc_exp_dend",
#                                           "synapse": "third_factor_stdp_synapse",
#                                           "post_ports": ["post_spikes",
#       http://localhost:8888/notebooks/doc/tutorials/sequences/sequences.ipynb#            
#["I_post_dend", "I_dend"]]}]}

# if not NESTTools.detect_nest_version().startswith("v2"):
#     codegen_opts["neuron_parent_class"] = "StructuralPlasticityNode"
#     codegen_opts["neuron_parent_class_include"] = "structural_plasticity_node.h"

# generate the "jit" model (co-generated neuron and synapse), that does not rely on ArchivingNode
# files = [os.path.join("models", "neurons", "iaf_psc_exp_dend_neuron.nestml"),
#          os.path.join("models", "synapses", "third_factor_stdp_synapse.nestml")]
# input_path = [os.path.realpath(os.path.join(os.path.dirname(__file__), os.path.join(
#     os.pardir, os.pardir, s))) for s in files]
# generate_nest_target(input_path=input_path,
#                      target_path="/tmp/nestml-jit",
#                      logging_level="INFO",
#                      module_name="nestml_jit_module",
#                      codegen_opts=codegen_opts)
#nest.Install("nestml_jit_module")

# generate and build code
"""

module_name, neuron_model_name, synapse_model_name = \
    NESTCodeGeneratorUtils.generate_code_for("../../../doc/tutorials/sequences/iaf_psc_exp_nonlineardendrite_neuron.nestml",
                                             "../../../models/synapses/stdsp_synapse.nestml",
                                             logging_level="DEBUG",
                                             post_ports=["post_spikes", ["dAP_trace", "dAP_trace"]])
"""

module_name = "nestml_d76233a4d30f455c84b5b585e84208a2_module"
neuron_model_name = "iaf_psc_exp_nonlineardendrited76233a4d30f455c84b5b585e84208a2_neuron_nestml__with_stdspd76233a4d30f455c84b5b585e84208a2_synapse_nestml"
synapse_model_name = "stdspd76233a4d30f455c84b5b585e84208a2_synapse_nestml__with_iaf_psc_exp_nonlineardendrited76233a4d30f455c84b5b585e84208a2_neuron_nestml"


"""# JUST THE NEURON MODEL
module_name, neuron_model_name = \
    NESTCodeGeneratorUtils.generate_code_for("../../../doc/tutorials/sequences/iaf_psc_exp_nonlineardendrite_neuron.nestml",
                                             logging_level="DEBUG")"""


# load dynamic library (NEST extension module) into NEST kernel
nest.Install(module_name)

Automatic pdb calling has been turned ON
[1,GLOBAL, INFO]: List of files that will be processed:
[2,GLOBAL, INFO]: /home/charl/julich/nestml-fork-clopath_synapse/nestml/doc/tutorials/sequences/iaf_psc_exp_nonlineardendrite2817e88e01de42069addea0bb3524aa3_neuron.nestml
[3,GLOBAL, INFO]: Target platform code will be generated in directory: '/home/charl/julich/nestml-fork-clopath_synapse/nestml/doc/tutorials/sequences/target'
[4,GLOBAL, INFO]: Target platform code will be installed in directory: '/tmp/nestml_target_5b0k1g20'

              -- N E S T --
  Copyright (C) 2004 The NEST Initiative

 Version: 3.6.0-post0.dev0
 Built: Feb  5 2024 05:39:31

 This program is provided AS IS and comes with
 NO WARRANTY. See the file LICENSE for details.

 Problems or suggestions?
   Visit https://www.nest-simulator.org

 Type 'nest.help()' to find out more about NEST.

[5,GLOBAL, INFO]: The NEST Simulator version was automatically detected as: master
[6,GLOBAL, INFO]: Given template root path is no

INFO:Analysing input:
INFO:{
    "dynamics": [
        {
            "expression": "V_m' = (-(V_m - E_L)) / tau_m + (I_kernel1__X__I_1 * 1.0 + (I_kernel2__X__I_2 * 1.0) + I_kernel3__X__I_3 * 1.0 + I_e) / C_m",
            "initial_values": {
                "V_m": "0"
            }
        },
        {
            "expression": "dAP_trace' = (-dAP_trace) / tau_h",
            "initial_values": {
                "dAP_trace": "0"
            }
        },
        {
            "expression": "I_kernel1__X__I_1 = exp(-1 / tau_syn1 * t)",
            "initial_values": {}
        },
        {
            "expression": "I_kernel2__X__I_2 = (e / tau_syn2) * t * exp(-t / tau_syn2)",
            "initial_values": {}
        },
        {
            "expression": "I_kernel3__X__I_3 = exp(-1 / tau_syn3 * t)",
            "initial_values": {}
        }
    ],
    "options": {
        "output_timestep_symbol": "__h"
    },
    "parameters": {
        "C_m": "250",
        "E_L": "0",
        "I_e": "

[14,GLOBAL, INFO]: Analysing/transforming model 'iaf_psc_exp_nonlineardendrite2817e88e01de42069addea0bb3524aa3_neuron_nestml'
[15,iaf_psc_exp_nonlineardendrite2817e88e01de42069addea0bb3524aa3_neuron_nestml, INFO, [1:0;93:0]]: Starts processing of the model 'iaf_psc_exp_nonlineardendrite2817e88e01de42069addea0bb3524aa3_neuron_nestml'


DEBUG:	checking whether shape definition is satisfied...
DEBUG:Shape satisfies ODE of order = 2
DEBUG:Created Shape with symbol I_kernel2__X__I_2, derivative_factors = Matrix([[-1/tau_syn2**2], [-2/tau_syn2]]), inhom_term = 0.0, nonlin_term = 0.0
INFO:Shape I_kernel2__X__I_2: reconstituting expression -I_kernel2__X__I_2/tau_syn2**2 - 2*I_kernel2__X__I_2__d/tau_syn2
INFO:
Processing function-of-time shape "I_kernel3__X__I_3" with defining expression = "exp(-t/tau_syn3)"
DEBUG:Found t: 0
DEBUG:	Finding ode for order 1...
DEBUG:Shape satisfies ODE of order = 1
DEBUG:Created Shape with symbol I_kernel3__X__I_3, derivative_factors = [-1/tau_syn3], inhom_term = 0.0, nonlin_term = 0.0
INFO:Shape I_kernel3__X__I_3: reconstituting expression -I_kernel3__X__I_3/tau_syn3
INFO:All known variables: [V_m, dAP_trace, I_kernel1__X__I_1, I_kernel2__X__I_2, I_kernel2__X__I_2', I_kernel3__X__I_3], all parameters used in ODEs: {C_m, tau_syn1, tau_syn2, tau_syn3, tau_m, E_L, tau_h, I_e}
INFO:
Processing di

DEBUG:	nonlinear term: 0.0
INFO:Shape I_kernel3__X__I_3: reconstituting expression -I_kernel3__X__I_3/tau_syn3
DEBUG:Splitting expression -I_kernel3__X__I_3/tau_syn3 (symbols [V_m, dAP_trace, I_kernel1__X__I_1, I_kernel2__X__I_2, I_kernel2__X__I_2__d, I_kernel3__X__I_3])
DEBUG:	linear factors: Matrix([[0], [0], [0], [0], [0], [-1/tau_syn3]])
DEBUG:	inhomogeneous term: 0.0
DEBUG:	nonlinear term: 0.0
INFO:Saving dependency graph plot to /tmp/ode_dependency_graph_analytically_solvable_before_propagated.dot
DEBUG:os.makedirs('/tmp')
DEBUG:write lines to '/tmp/ode_dependency_graph_analytically_solvable_before_propagated.dot'
DEBUG:run [PosixPath('dot'), '-Kdot', '-Tpdf', '-O', 'ode_dependency_graph_analytically_solvable_before_propagated.dot']
INFO:Saving dependency graph plot to /tmp/ode_dependency_graph_analytically_solvable.dot
DEBUG:os.makedirs('/tmp')
DEBUG:write lines to '/tmp/ode_dependency_graph_analytically_solvable.dot'
DEBUG:run [PosixPath('dot'), '-Kdot', '-Tpdf', '-O', 'ode_dep

[16,iaf_psc_exp_nonlineardendrite2817e88e01de42069addea0bb3524aa3_neuron_nestml, DEBUG, [1:0;93:0]]: Start building symbol table!
[17,iaf_psc_exp_nonlineardendrite2817e88e01de42069addea0bb3524aa3_neuron_nestml, INFO, [69:29;69:29]]: Implicit casting from (compatible) type 'pA' to 'real'.
[18,iaf_psc_exp_nonlineardendrite2817e88e01de42069addea0bb3524aa3_neuron_nestml, INFO, [72:29;72:31]]: Implicit casting from (compatible) type 'pA' to 'real'.
[19,iaf_psc_exp_nonlineardendrite2817e88e01de42069addea0bb3524aa3_neuron_nestml, INFO, [77:25;77:25]]: Implicit casting from (compatible) type 'pA' to 'real'.
[20,iaf_psc_exp_nonlineardendrite2817e88e01de42069addea0bb3524aa3_neuron_nestml, INFO, [86:25;86:27]]: Implicit casting from (compatible) type 'pA' to 'real'.
[21,iaf_psc_exp_nonlineardendrite2817e88e01de42069addea0bb3524aa3_neuron_nestml, INFO, [93:21;93:23]]: Implicit casting from (compatible) type 'pA' to 'real'.
[22,GLOBAL, INFO]: Rendering template /home/charl/julich/nestml-fork-clopat

Now, the NESTML model is ready to be used in a simulation.

In [3]:
import parameters as para
import numpy as np

p = para.ParameterSpace({})

DELAY = 0.1

p['dt'] = 0.1                                  # simulation time resolution (ms)
p['print_simulation_progress'] = False         # print the time progress.
p['n_threads'] = 4                             # number of threads per MPI process 


# data path dict
p['data_path'] = {}
p['data_path']['data_root_path'] = 'data'
p['data_path']['project_name'] = 'sequence_learning_performance'
p['data_path']['parameterspace_label'] = 'effect_dAP_firing_times'

# neuron parameters of the excitatory neurons
p['soma_model'] = neuron_model_name
p['soma_params'] = {}
p['soma_params']['C_m'] = 250.        # membrane capacitance (pF)
p['soma_params']['E_L'] = 0.          # resting membrane potential (mV)
# p['soma_params']['I_e'] = 0.        # external DC currents (pA)
p['soma_params']['V_m'] = 0.          # initial potential (mV)
p['soma_params']['V_reset'] = 0.      # reset potential (mV)
p['soma_params']['V_th'] = 20.        # spike threshold (mV)
p['soma_params']['t_ref'] = 10.       # refractory period
p['soma_params']['tau_m'] = 10.       # membrane time constant (ms)
p['soma_params']['tau_syn1'] = 2.     # synaptic time constant: external input (receptor 1)
p['soma_params']['tau_syn2'] = 5.     # synaptic time constant: dendrtic input (receptor 2)
p['soma_params']['tau_syn3'] = 1.     # synaptic time constant: inhibitory input (receptor 3)
# dendritic action potential
p['soma_params']['I_p'] = 200. # current clamp value for I_dAP during a dendritic action potenti
p['soma_params']['tau_dAP'] = 60.       # time window over which the dendritic current clamp is active
p['soma_params']['theta_dAP'] = 59.        # current threshold for a dendritic action potential

p['soma_params']['I_dend_incr'] = 2.71 / 10e-3


p['fixed_somatic_delay'] = 2          # this is an approximate time of how long it takes the soma to fire
                                      # upon receiving an external stimulus 

# neuron parameters for the inhibitory neuron
p['inhibit_model'] = 'iaf_psc_exp'
p['inhibit_params'] = {}
p['inhibit_params']['C_m'] = 250.         # membrane capacitance (pF)
p['inhibit_params']['E_L'] = 0.           # resting membrane potential (mV)
p['inhibit_params']['I_e'] = 0.           # external DC currents (pA)
p['inhibit_params']['V_m'] = 0.           # initial potential (mV)
p['inhibit_params']['V_reset'] = 0.       # reset potential (mV)
p['inhibit_params']['V_th'] = 15.         # spike threshold (mV)
p['inhibit_params']['t_ref'] = 2.0        # refractory period
p['inhibit_params']['tau_m'] = 5.         # membrane time constant (ms)
p['inhibit_params']['tau_syn_ex'] = .5    # synaptic time constant of an excitatory input (ms) 
p['inhibit_params']['tau_syn_in'] = 1.65  # synaptic time constant of an inhibitory input (ms)

# synaptic parameters
p['J_EX_psp'] = 1.1 * p['soma_params']['V_th']     # somatic PSP as a response to an external input
p['J_IE_psp'] = 1.2 * p['inhibit_params']['V_th']  # inhibitory PSP as a response to an input from E neuron
p['J_EI_psp'] = -2 * p['soma_params']['V_th']      # somatic PSP as a response to an inhibitory input
p['convergence'] = 5
p['pattern_size'] = 20

# parameters for ee synapses (stdsp)
p['syn_dict_ee'] = {}
p['p_min'] = 0.
p['p_max'] = 8.
p['calibration'] = 40.
p['syn_dict_ee']['weight'] = 0.01                    # synaptic weight
p['syn_dict_ee']['synapse_model'] = synapse_model_name  # synapse model
p['syn_dict_ee']['permanence_threshold'] = 10.                    # synapse maturity threshold
p['syn_dict_ee']['tau_pre_trace'] = 20.                   # plasticity time constant (potentiation)
p['syn_dict_ee']['delay'] = 2.                       # dendritic delay 
p['syn_dict_ee']['receptor_type'] = 2                # receptor corresponding to the dendritic input
p['syn_dict_ee']['lambda_plus'] = 0.05 #0.1                     # potentiation rate
p['syn_dict_ee']['zt'] = 1.                          # target dAP trace [pA]
p['syn_dict_ee']['lambda_h'] = 0.01                        # homeostasis rate
p['syn_dict_ee']['Wmax'] = 1.1 * p['soma_params']['theta_dAP'] / p['convergence']   # Maximum allowed weight
p['syn_dict_ee']['permanence_max'] = 20.                       # Maximum allowed permanence
p['syn_dict_ee']['permanence_min'] = 1.                        # Minimum allowed permanence
p['syn_dict_ee']['lambda_minus'] = 0.004

# parameters of EX synapses (external to soma of E neurons)
p['conn_dict_ex'] = {}
p['syn_dict_ex'] = {}
p['syn_dict_ex']['receptor_type'] = 1                    # receptor corresponding to external input
p['syn_dict_ex']['delay'] = DELAY                        # dendritic delay
p['conn_dict_ex']['rule'] = 'all_to_all'                 # connection rule

# parameters of EdX synapses (external to dendrite of E neurons) 
p['conn_dict_edx'] = {}
p['syn_dict_edx'] = {}
p['syn_dict_edx']['receptor_type'] = 2                    # receptor corresponding to the dendritic input
p['syn_dict_edx']['delay'] = DELAY                        # dendritic delay
p['syn_dict_edx']['weight'] = 10.4 * p['soma_params']['theta_dAP']
p['conn_dict_edx']['rule'] = 'fixed_outdegree'            # connection rule
p['conn_dict_edx']['outdegree'] = p['pattern_size'] + 1   # outdegree

# parameters for IE synapses 
p['syn_dict_ie'] = {}
p['conn_dict_ie'] = {}
p['syn_dict_ie']['synapse_model'] = 'static_synapse'     # synapse model
p['syn_dict_ie']['delay'] = DELAY                        # dendritic delay
p['conn_dict_ie']['rule'] = 'fixed_indegree'             # connection rule
p['conn_dict_ie']['indegree'] = 5                        # indegree 

# parameters for EI synapses
p['syn_dict_ei'] = {}
p['conn_dict_ei'] = {}
p['syn_dict_ei']['synapse_model'] = 'static_synapse'     # synapse model
p['syn_dict_ei']['delay'] = DELAY                        # dendritic delay
p['syn_dict_ei']['receptor_type'] = 3                    # receptor corresponding to the inhibitory input  
p['conn_dict_ei']['rule'] = 'fixed_indegree'             # connection rule
p['conn_dict_ei']['indegree'] = 20                       # indegree





In [4]:
def psp_max_2_psc_max(psp_max, tau_m, tau_s, R_m):
    """Compute the PSC amplitude (pA) injected to get a certain PSP maximum (mV) for LIF with exponential PSCs

    Parameters
    ----------
    psp_max: float
             Maximum postsynaptic pontential
    tau_m:   float
             Membrane time constant (ms).
    tau_s:   float
             Synaptic time constant (ms).
    R_m:     float
             Membrane resistance (Gohm).

    Returns
    -------
    float
        PSC amplitude (pA).
    """

    return psp_max / (
            R_m * tau_s / (tau_s - tau_m) * (
            (tau_m / tau_s) ** (-tau_m / (tau_m - tau_s)) -
            (tau_m / tau_s) ** (-tau_s / (tau_m - tau_s))
    )
    )


In [5]:
params = p
params['R_m_soma'] = params['soma_params']['tau_m'] / params['soma_params']['C_m']
params['R_m_inhibit'] = params['inhibit_params']['tau_m'] / params['inhibit_params']['C_m']
params['syn_dict_ex']['weight'] = psp_max_2_psc_max(params['J_EX_psp'], 
                                                           params['soma_params']['tau_m'], 
                                                           params['soma_params']['tau_syn1'], 
                                                           params['R_m_soma'])
params['syn_dict_ie']['weight'] = psp_max_2_psc_max(params['J_IE_psp'], 
                                                           params['inhibit_params']['tau_m'], 
                                                           params['inhibit_params']['tau_syn_ex'], 
                                                           params['R_m_inhibit'])
params['syn_dict_ei']['weight'] = psp_max_2_psc_max(params['J_EI_psp'], 
                                                           params['soma_params']['tau_m'], 
                                                           params['soma_params']['tau_syn3'], 
                                                           params['R_m_soma'])

soma_excitation_time = 25.
dendrite_excitation_time = 3.

In [ ]:
print("Running simulations")
data = {}
for i, name in enumerate(['ff', 'dendrite', 'ff_dendrite']): 
    print("Running experiment type: " + name)
    # init kernel
    seed = 1
    nest.ResetKernel()
    nest.set_verbosity("M_ALL")
    nest.SetKernelStatus({
        'resolution': params['dt'],
        'print_time': params['print_simulation_progress'],
        'local_num_threads': params['n_threads'],
        'rng_seed': seed
    })

    data[name] = {}

    #############################
    # create and connect neurons
    # ---------------------------

    # create excitatory population
    exc_neuron = nest.Create(params['soma_model'], params=params['soma_params'])

    # create inhibitory population
    inh_neuron = nest.Create(params['inhibit_model'], params=params['inhibit_params'])

    # connect inhibition
    nest.Connect(exc_neuron, inh_neuron, syn_spec=params['syn_dict_ie'])
    nest.Connect(inh_neuron, exc_neuron, syn_spec=params['syn_dict_ei'])

    ######################
    # Input stream/stimuli
    #---------------------
    input_excitation = nest.Create('spike_generator', params={'spike_times':[soma_excitation_time]})
    dendrite_excitation_1 = nest.Create('spike_generator', params={'spike_times':[dendrite_excitation_time]})
    #dendrite_excitation_2 = nest.Create('spike_generator', params={'spike_times':[7.]})
    inhibition_excitation = nest.Create('spike_generator', params={'spike_times':[10.]})

    # excitation soma feedforward
    if name == 'ff' or name == 'ff_dendrite':
        nest.Connect(input_excitation, exc_neuron, syn_spec={'receptor_type': 1, 
                                                             'weight': params['syn_dict_ex']['weight'], 
                                                             'delay': params['syn_dict_ex']['delay']})

    # excitation dendrite 
    if name == 'dendrite' or name == 'ff_dendrite':
        nest.Connect(dendrite_excitation_1, exc_neuron, syn_spec={'receptor_type': 2, 
                                                                  'weight': params['syn_dict_edx']['weight'], 
                                                                  'delay': params['syn_dict_edx']['delay']})

    # record voltage inhibitory neuron 
    vm_inh = nest.Create('voltmeter', params={'record_from': ['V_m'], 'interval': 0.1})
    nest.Connect(vm_inh, inh_neuron)

    # record voltage soma
    vm_exc = nest.Create('voltmeter', params={'record_from': ['V_m'], 'interval': 0.1})
    nest.Connect(vm_exc, exc_neuron)

    active_dendrite_exc_mm = nest.Create('multimeter', params={'record_from': ['active_dendrite_readout', 'I_dend'], 'interval': 0.1})
    nest.Connect(active_dendrite_exc_mm, exc_neuron)

    # record spikes
    sd = nest.Create('spike_recorder')
    nest.Connect(exc_neuron, sd)

    # record inh spikes
    sd_inh = nest.Create('spike_recorder')
    nest.Connect(inh_neuron, sd_inh)

    print('### simulating network')
    #nest.Simulate(100.)
    nest.Prepare()
    nest.Run(100.)

    voltage_soma = nest.GetStatus(vm_exc)[0]['events']  
    active_dendrite = nest.GetStatus(active_dendrite_exc_mm)[0]['events']
    voltage_inhibit = nest.GetStatus(vm_inh)[0]['events'] 
    spikes_soma = nest.GetStatus(sd)[0]['events'] 
    spikes_inh = nest.GetStatus(sd_inh)[0]['events'] 

    data[name]['exc'] = voltage_soma 
    data[name]['exc_active_dendrite'] = active_dendrite 
    data[name]['inh'] = voltage_inhibit
    data[name]['spikes_exc'] = spikes_soma
    data[name]['spikes_inh'] = spikes_inh

## Plotting

In [ ]:
def position_excitation_arrows(ax, soma_time, dendrite_time):

    arrow_width = 1.8
    arrow_height = 1.8
    y = -2.3
    
    # plot excitation arrows for panel A
    x = soma_time - arrow_width/2 
    pos = [x, y]
    X = np.array([pos, [pos[0]+arrow_width, pos[1]], [pos[0]+arrow_width/2, pos[1]+arrow_height]])
    t1 = plt.Polygon(X, color=color_somatic_input)
    ax[0].add_patch(t1)

    # plot excitation arrows for panel B
    x = dendrite_time - arrow_width/2 
    pos = [x, y]
    X = np.array([pos, [pos[0]+arrow_width, pos[1]], [pos[0]+arrow_width/2, pos[1]+arrow_height]])
    t1 = plt.Polygon(X, color=color_dAP_input)
    ax[1].add_patch(t1)

    # plot excitation arrows for panel C
    x = dendrite_time - arrow_width/2 
    pos = [x, y]
    X = np.array([pos, [pos[0]+arrow_width, pos[1]], [pos[0]+arrow_width/2, pos[1]+arrow_height]])
    t1 = plt.Polygon(X, color=color_dAP_input)
    ax[2].add_patch(t1)

    x = soma_time - arrow_width/2 
    pos = [x, y]
    X = np.array([pos, [pos[0]+arrow_width, pos[1]], [pos[0]+arrow_width/2, pos[1]+arrow_height]])
    t1 = plt.Polygon(X, color=color_somatic_input)
    ax[2].add_patch(t1)


color_dAP_input = '#8e7c42ff'
#color_somatic_input = '#0000ffff'
color_somatic_input = '#4581a7ff'
color_soma = '#000000ff'
color_dAP = '#00B4BE' 
color_inhibit = '#808080ff'  
color_hrl = 'black'

#color_somatic_spike = '#ff0000ff'
color_somatic_spike = color_soma
color_inh_spike = color_inhibit
ms_spike = 7
mew_spike = 1.5
lw_vtheta = 0.5
lw_dAP = 1.5
lw_s = 1.5
lw_i = 1.5

# plot settings 
fig_size = (6., 5)
ymin = -4
ymax = params['soma_params']['V_th'] + 4
xmin = 0  
xmax = 85
label_pos = (-0.18, 1.)
panel_labels = ['A', 'B', 'C']
v_th=params['soma_params']['V_th'] 
time_dAP = 10
soma_excitation_time = 25.
dendrite_excitation_time = 3.

# set up the figure frame
fig = plt.figure()
gs = mpl.gridspec.GridSpec(5, 1, height_ratios=[15,15,15,5,6], bottom=0.1, right=0.95, top=0.93, wspace=0., hspace=0.1)
left, bottom, width, height = [0.4, 0.1, 0.2, 0.2]
axes = []



for i, name in enumerate(['ff', 'dendrite', 'ff_dendrite']):

    #ax = fig.add_subplot(gs[i,0])
    ax = plt.subplot(gs[i,0])
    ax.text(label_pos[0], label_pos[1], panel_labels[i], transform=ax.transAxes, horizontalalignment='center', verticalalignment='center', size=10, weight='bold')
    ax.plot(data[name]['exc']['times'], data[name]['exc']['V_m'], lw=lw_s, color=color_soma, zorder=2, label='excitatory neuron')  
    
    
    ax.plot(data[name]['exc_active_dendrite']['times'], data[name]['exc_active_dendrite']['active_dendrite_readout'], lw=lw_s, color=color_dAP)  
    
    ax_ = ax.twinx()
    ax_.plot(data[name]['exc_active_dendrite']['times'], data[name]['exc_active_dendrite']['I_dend'], lw=lw_s, color="red", label="I_dend")  
    ax_.plot((0., np.amax(data[name]['exc_active_dendrite']['times'])), 2*[p['soma_params']['theta_dAP']], c="red", linestyle=':')
             
    ax.plot(data[name]['spikes_exc']['times'], (v_th+2)*np.ones(len(data[name]['spikes_exc']['times'])), '|', c=color_somatic_spike, ms=ms_spike, mew=mew_spike)
    ax.plot(data[name]['spikes_inh']['times'], (v_th+2)*np.ones(len(data[name]['spikes_inh']['times'])), '|', c=color_inh_spike, ms=ms_spike, mew=mew_spike)
    ax.legend()
 
    # add dendritic action potential bar manually
    if name == 'dendrite': 
        ax.hlines(v_th+2, time_dAP, time_dAP+params['soma_params']['tau_dAP'], lw=lw_dAP, color=color_dAP)

    if name == 'ff_dendrite': 
        ax.hlines(v_th+2, time_dAP, data[name]['spikes_exc']['times'][0], lw=lw_dAP, color=color_dAP)

    # clamp voltage if doesn't reach the firing threshold
    if name == 'ff' or name == 'ff_dendrite': 
        max_volt = max(data[name]['inh']['V_m']) 
        max_volt_ind = np.where(data[name]['inh']['V_m']==max_volt)[0]
        data[name]['inh']['V_m'][max_volt_ind] = 20

    ax.plot(data[name]['inh']['times'], data[name]['inh']['V_m'], lw=lw_i, color=color_inhibit, zorder=1, label='inhibitory neuron') 
    ax.set_ylim([ymin, ymax])
    ax.set_xlim([xmin, xmax])
    ax.hlines(v_th, xmin, xmax, lw=lw_vtheta, color=color_hrl, linestyle='--')

    axes.append(ax)

axes[1].set_ylabel('membrane potential (mV)')

# set position of arrows
position_excitation_arrows(axes, soma_excitation_time, dendrite_excitation_time)

axes[0].legend(loc='center right')
axes[0].set_yticklabels([])
axes[0].set_xticklabels([])
axes[1].set_xticklabels([])
axes[2].set_yticklabels([])
axes[2].set_xlabel('time (ms)')

########################################
# plt spikes of A and B
# --------------------------------------
ax = fig.add_subplot(gs[i+1,0])
plt.axis('off')

ax = plt.subplot(gs[i+2,0])
ax.text(label_pos[0], label_pos[1], 'D', transform=ax.transAxes, horizontalalignment='center', verticalalignment='center', size=10, weight='bold')

xmin_d=25.6
xmax_d=29

ymin_d=0
ymax_d=10

name = 'ff'
ax.plot(data[name]['spikes_exc']['times'], (3*ymax_d/4)*np.ones(len(data[name]['spikes_exc']['times'])), '|', c=color_somatic_spike, ms=ms_spike, mew=mew_spike)
ax.plot(data[name]['spikes_inh']['times'], (3*ymax_d/4)*np.ones(len(data[name]['spikes_inh']['times'])), '|', c=color_inh_spike, ms=ms_spike, mew=mew_spike)

name = 'ff_dendrite'
ax.plot(data[name]['spikes_exc']['times'], (ymax_d/4)*np.ones(len(data[name]['spikes_exc']['times'])), '|', c=color_somatic_spike, ms=ms_spike, mew=mew_spike)
ax.plot(data[name]['spikes_inh']['times'], (ymax_d/4)*np.ones(len(data[name]['spikes_inh']['times'])), '|', c=color_inh_spike, ms=ms_spike, mew=mew_spike)
ax.hlines(ymax_d/2, xmin, xmax, lw=0.5, color=color_hrl, linestyles='solid')

ax.set_yticklabels([])
ax.tick_params(left=False)
ax.set_ylim([ymin_d, ymax_d])
ax.set_xlim([xmin_d, xmax_d])
ax.set_xlabel('time (ms)')

ax.text(xmin_d+0.05, (3*ymax_d/4)-1, 'A', size=8, weight='bold')
ax.text(xmin_d+0.05, (ymax_d/4)-1, 'C', size=8, weight='bold')

############################################################
# add lines between the subplots showing the zoomed in area
# ----------------------------------------------------------
xy_C = (xmin_d,ymin)
xy_D = (xmin_d,ymax_d)
con = mpl.patches.ConnectionPatch(xyA=xy_C, xyB=xy_D, coordsA='data', coordsB='data', axesA=axes[-1], axesB=ax, color='grey', linestyle='dotted')
ax.add_artist(con)

xy_C = (xmax_d,ymin)
xy_D = (xmax_d,ymax_d)
con = mpl.patches.ConnectionPatch(xyA=xy_C, xyB=xy_D, coordsA='data', coordsB='data', axesA=axes[-1], axesB=ax, color='grey', linestyle='dotted')
ax.add_artist(con)

plt.savefig("/tmp/sequences1.png")

## Plasticity dynamics dependence on dAP

In [ ]:

DELAY = 0.1

p = para.ParameterSpace({})

p['dt'] = 0.1                                  # simulation time resolution (ms)
p['print_simulation_progress'] = True         # print the time progress.
p['n_threads'] = 1                             # number of threads per MPI process 

# data path dict
p['data_path'] = {}
p['data_path']['data_root_path'] = 'data'
p['data_path']['project_name'] = 'sequence_learning_performance'
p['data_path']['parameterspace_label'] = 'plasticity_dynamics'

# neuron parameters of the excitatory neurons
p['soma_model'] = neuron_model_name
p['soma_params'] = {}
p['soma_params']['C_m'] = 250.        # membrane capacitance (pF)
p['soma_params']['E_L'] = 0.          # resting membrane potential (mV)
# p['soma_params']['I_e'] = 0.        # external DC currents (pA)
p['soma_params']['V_m'] = 0.          # initial potential (mV)
p['soma_params']['V_reset'] = 0.      # reset potential (mV)
p['soma_params']['V_th'] = 20.        # spike threshold (mV)
p['soma_params']['t_ref'] = 10.       # refractory period
p['soma_params']['tau_m'] = 10.       # membrane time constant (ms)
p['soma_params']['tau_syn1'] = 2.     # synaptic time constant: external input (receptor 1)
p['soma_params']['tau_syn2'] = 5.     # synaptic time constant: dendrtic input (receptor 2)
p['soma_params']['tau_syn3'] = 1.     # synaptic time constant: inhibitory input (receptor 3)
# dendritic action potential
p['soma_params']['I_p'] = 200. # current clamp value for I_dAP during a dendritic action potenti
p['soma_params']['tau_dAP'] = 60.       # time window over which the dendritic current clamp is active
p['soma_params']['theta_dAP'] = 59.        # current threshold for a dendritic action potential
p['fixed_somatic_delay'] = 2          # this is an approximate time of how long it takes the soma to fire
                                      # upon receiving an external stimulus 

# synaptic parameters
p['J_EX_psp'] = 1.1 * p['soma_params']['V_th']     # somatic PSP as a response to an external input
p['convergence'] = 5
p['pattern_size'] = 20       # sparse set of active neurons per subpopulation

# parameters for ee synapses (stdsp)
p['syn_dict_ee'] = {}
p['p_min'] = 0.
p['p_max'] = 8.
p['calibration'] = 0.
p['syn_dict_ee']['weight'] = 0.01                    # synaptic weight
p['syn_dict_ee']['synapse_model'] = synapse_model_name  # synapse model
p['syn_dict_ee']['permanence_threshold'] = 10.                    # synapse maturity threshold
p['syn_dict_ee']['tau_pre_trace'] = 20.                   # plasticity time constant (potentiation)
p['syn_dict_ee']['delay'] = 2.                       # dendritic delay 
p['syn_dict_ee']['receptor_type'] = 2                # receptor corresponding to the dendritic input
p['syn_dict_ee']['lambda_plus'] = 0.08                     # potentiation rate
p['syn_dict_ee']['zt'] = 1.                          # target dAP trace [pA]
p['syn_dict_ee']['lambda_h'] = 0.014                        # homeostasis rate
p['syn_dict_ee']['Wmax'] = 1.1 * p['soma_params']['theta_dAP'] / p['convergence']   # Maximum allowed weight
p['syn_dict_ee']['permanence_max'] = 20.                       # Maximum allowed permanence
p['syn_dict_ee']['permanence_min'] = 1.                        # Minimum allowed permanence
p['syn_dict_ee']['lambda_minus'] = 0.0015

# parameters of EX synapses (external to soma of E neurons)
p['conn_dict_ex'] = {}
p['syn_dict_ex'] = {}
p['syn_dict_ex']['receptor_type'] = 1                    # receptor corresponding to external input
p['syn_dict_ex']['delay'] = DELAY                        # dendritic delay
p['conn_dict_ex']['rule'] = 'all_to_all'                 # connection rule

## stimulus parameters
p['DeltaT'] = 40.                               # inter-stimulus interval

In [ ]:
nest.ResetKernel()
nest.set_verbosity("M_ALL")
nest.SetKernelStatus({
    'resolution': params['dt'],
    'print_time': params['print_simulation_progress'],
    'local_num_threads': params['n_threads'],
    'rng_seed': seed
})

In [ ]:
# neuron parameters
params = p

neuron_1 = nest.Create(params['soma_model'], params=params['soma_params'])
neuron_2 = nest.Create(params['soma_model'], params=params['soma_params'])

# connect two neurons
nest.Connect(neuron_1, neuron_2, syn_spec=params['syn_dict_ee'])

# creation of spike generator
time_neuron_1 = 10.
time_neuron_2 = time_neuron_1 + params['DeltaT']

training_steps = 120
between_exc = 5*params['DeltaT']

times_neuron_1 = [time_neuron_1+i*between_exc for i in range(training_steps)]
times_neuron_2 = [time_neuron_2+i*between_exc for i in range(training_steps)]#[:10]

# create the spike generators 
# disable spike generator for the interval 'dis', to see the affect of stpd
dis = 20
spike_generator_1 = nest.Create('spike_generator', params={'spike_times': times_neuron_1})
spike_generator_2 = nest.Create('spike_generator', params={'spike_times': times_neuron_2})

# connect the spike generator 

params['R_m_soma'] = params['soma_params']['tau_m'] / params['soma_params']['C_m']
params['syn_dict_ex']['weight'] = psp_max_2_psc_max(params['J_EX_psp'], 
                                                           params['soma_params']['tau_m'], 
                                                           params['soma_params']['tau_syn1'], 
                                                           params['R_m_soma'])

syn_dict_ff = {'receptor_type': 1, 'weight': params['syn_dict_ex']['weight'], 'delay': params['syn_dict_ex']['delay']}
nest.Connect(spike_generator_1, neuron_1, syn_spec=syn_dict_ff)
nest.Connect(spike_generator_2, neuron_2, syn_spec=syn_dict_ff)

# record voltage neuron 1, neuron 2
dap_mm_1 = nest.Create('multimeter', {"record_from": ["dAP_trace"]})
nest.Connect(dap_mm_1, neuron_1)

dap_mm_2 = nest.Create('multimeter', {"record_from": ["dAP_trace"]})
nest.Connect(dap_mm_2, neuron_2)

vm_1 = nest.Create('voltmeter')
vm_2 = nest.Create('voltmeter')
nest.Connect(vm_1, neuron_1)
nest.Connect(vm_2, neuron_2)


sd_1 = nest.Create('spike_recorder')
nest.Connect(neuron_1, sd_1)

sd_2 = nest.Create('spike_recorder')
nest.Connect(neuron_2, sd_2)


synColl = nest.GetConnections(synapse_model=synapse_model_name)
assert len(synColl) == 1

print('\n simulate')
zs = [0.,1.,2.]
weights_cs = []
permanences_cs = []

for z in zs:

    weights = []
    permanences = []
    last_sim_time = 0
    
    spike_generator_1.origin = nest.GetKernelStatus('biological_time')
    spike_generator_2.origin = nest.GetKernelStatus('biological_time')
    
    # connect two neurons
    synColl.set({'permanence': 1.}) 

    for i in range(training_steps):

        # change toward using the weight recorder, example:
        #wr = nest.Create('weight_recorder')
        #nest.CopyModel('stdp_synapse', 'stdp_synapse_rec', {'weight_recorder': wr})

        nest.SetStatus(neuron_1, {'dAP_trace':z})
        nest.SetStatus(neuron_2, {'dAP_trace':z})

        # simulate the network
        sim_time = times_neuron_1[i] - last_sim_time 
        nest.Simulate(sim_time)
        last_sim_time = times_neuron_1[i]

        w_after = synColl.weight
        p_after = synColl.permanence
        weights.append(w_after)
        permanences.append(p_after)

        
    fig, ax = plt.subplots(figsize=(24,6))
    ax.plot(nest.GetStatus(vm_1)[0]['events']["times"], nest.GetStatus(vm_1)[0]['events']["V_m"], label="vm1")
    max_V_m = np.amax(nest.GetStatus(vm_1)[0]['events']["V_m"])
    ax.scatter(nest.GetStatus(sd_1)[0]['events']['times'], max_V_m * np.ones_like(nest.GetStatus(sd_1)[0]['events']['times']))
    ax.plot(nest.GetStatus(vm_2)[0]['events']["times"], nest.GetStatus(vm_2)[0]['events']["V_m"], label="vm2")
    ax.scatter(nest.GetStatus(sd_2)[0]['events']['times'], max_V_m * np.ones_like(nest.GetStatus(sd_2)[0]['events']['times']))
    ax.legend()
    ax_ = ax.twinx()
    ax_.plot(nest.GetStatus(dap_mm_1)[0]['events']["times"], nest.GetStatus(dap_mm_1)[0]['events']["dAP_trace"], label="dAP")
    ax_.plot(nest.GetStatus(dap_mm_2)[0]['events']["times"], nest.GetStatus(dap_mm_2)[0]['events']["dAP_trace"], label="dAP")
    
    fig.savefig("/tmp/foo" + str(z) + ".png")
        
    weights_cs.append(weights)
    permanences_cs.append(permanences)

# store postprocessed
params['zs'] = zs


In [ ]:
# load data
# ----------

path_dict = {} 
path_dict['data_root_path'] = 'data'
path_dict['project_name'] = 'sequence_learning_performance' 
path_dict['parameterspace_label'] = 'plasticity_dynamics'


# plot recorded data
# ------------------

# plot settings 
fig_size = (5.2, 2.)
plt.rcParams['font.size'] = 8
plt.rcParams['legend.fontsize'] = 6
plt.rcParams['figure.figsize'] = fig_size
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['text.usetex'] = False
ms = 0.5
alpha = 0.5
lw_hline = 1.

#################
# visualize data
# ---------------
gs = mpl.gridspec.GridSpec(1, 3, right=0.92, left=0.09, bottom=0.2, top=0.89, wspace=0.2, hspace=0.2)

# data for Ic=0
# -------------
ax1 = plt.subplot(gs[0,0])

training_steps = len(weights_cs[0])
num_pulses = np.arange(training_steps)
lns1 = ax1.plot(num_pulses, weights_cs[0], '-o', ms=ms, color='black', label=r'$J$')

#plt.ylabel('weight ($\mu$S)')
ax1.set_xlim(0, training_steps)
ax1.set_ylim(-1, params["syn_dict_ee"]['Wmax']+10)
#ax1.set_title(r'dAP rate $\nu_\mathsf{d}$=%0.1f' % zs[0])
ax1.set_title(r'$z$=%0.1f' % zs[0])
ax1.set_ylabel(r'weight $J$ (pA)')

ax2 = ax1.twinx()
lns2 = ax2.plot(num_pulses, permanences_cs[0], '-o', ms=ms, color='grey', alpha=alpha, label=r'$P$')
plt.hlines(params['syn_dict_ee']['permanence_threshold'], 0, training_steps, lw=lw_hline, color='grey', linestyles='dotted')
plt.hlines(params['syn_dict_ee']['permanence_max'], 0, training_steps, lw=lw_hline, color='grey', linestyles='dashed')

ax2.set_ylim(-1, params["syn_dict_ee"]['permanence_max']+2)
ax2.tick_params(axis='y', labelcolor='grey')
#ax2.set_yticklabels([])
ax2.set_yticks([])
ax2.spines['right'].set_color('grey')

# add legends
lns = [lns1[0],lns2[0]]
labs = [l.get_label() for l in lns]
ax1.legend(lns, labs, loc='lower right')

# data for Ic=1
# -------------
ax1 = plt.subplot(gs[0,1])

ax1.plot(num_pulses, weights_cs[1], '-o', ms=ms, color='black', label='weight')

ax1.set_ylim(-1, params["syn_dict_ee"]['Wmax']+10)
ax1.set_xlim(0, training_steps)
#ax1.set_title(r'dAP rate $\nu_\mathsf{d}$=%0.1f' % zs[1])
ax1.set_title(r'$z$=%0.1f' % zs[1])
ax1.set_xlabel('number of presynaptic-postsynaptic spike pairings')
ax1.set_yticks([])

ax2 = ax1.twinx()
ax2.plot(num_pulses, permanences_cs[1], '-o', ms=ms, color='grey', alpha=alpha, label='permanence')
plt.hlines(params['syn_dict_ee']['permanence_threshold'], 0, training_steps, lw=lw_hline, color='grey', linestyles='dotted')
plt.hlines(params['syn_dict_ee']['permanence_max'], 0, training_steps, lw=lw_hline, color='grey', linestyles='dashed')

ax2.set_ylim(-1, params["syn_dict_ee"]['permanence_max']+2)
ax2.tick_params(axis='y', labelcolor='grey')
ax2.set_yticks([])
ax2.spines['right'].set_color('grey')

# data for Ic=2
# -------------
ax1 = plt.subplot(gs[0,2])

ax1.plot(num_pulses, weights_cs[2], '-o', ms=ms, color='black', label='weight')

ax1.set_ylim(-1, params["syn_dict_ee"]['Wmax']+10)
ax1.set_xlim(0, training_steps)
#ax1.set_title(r'dAP rate $\nu_\mathsf{d}$=%0.1f' % zs[2])
ax1.set_title(r'$z$=%0.1f' % zs[2])
ax1.set_yticks([])

ax2 = ax1.twinx()
ax2.plot(num_pulses, permanences_cs[2], '-o', ms=ms, color='grey', alpha=alpha, label=r'$P$')
plt.hlines(params['syn_dict_ee']['permanence_threshold'], 0, training_steps, lw=lw_hline, color='grey', linestyles='dotted')
plt.hlines(params['syn_dict_ee']['permanence_max'], 0, training_steps, lw=lw_hline, color='grey', linestyles='dashed')

ax2.set_ylim(-1, params["syn_dict_ee"]['permanence_max']+2)
ax2.tick_params(axis='y', labelcolor='grey')
ax2.set_ylabel(r"permanence $P$", color="grey")
#ax2.spines['right'].set_color('grey')

print('---------------------------------')
path = '.'
fname = 'plasticity_dynamics'
#print("save %s/%s.pdf" % (path, fname))
#plt.savefig("/tmp/%s.pdf" % fname)
plt.savefig("/tmp/%s.png" % fname)



References
----------

[1] Bouhadjar Y, Wouters DJ, Diesmann M, Tetzlaff T (2022) Sequence learning, prediction, and replay in networks of spiking neurons. PLoS Comput Biol 18(6): e1010233. https://doi.org/10.1371/journal.pcbi.1010233

